# 歌詞創作 AI - LoRA 微調實戰

## 學習目標
- 掌握單一任務的 LoRA 微調流程
- 學會準備豐富的歌詞創作數據
- 學會使用 LoRA 策略微調 LLM
- 創建 Gradio Web UI 互動介面

## 歌詞創作特色
- **多元風格**：流行、民謠、搖滾、說唱等
- **情感豐富**：愛情、友情、勵志、懷舊等
- **結構完整**：主歌、副歌、橋段、結尾
- **押韻優美**：注重韻腳和節奏感

## 1. 環境設置與套件安裝

In [1]:
# 安裝必要套件
!pip install -q torch transformers peft datasets accelerate bitsandbytes gradio

import os
import json
import torch
import warnings
from datetime import datetime
from typing import Dict, List, Any
import gradio as gr

warnings.filterwarnings("ignore")

print("🎵 歌詞創作 AI 訓練環境")
print("=" * 40)
print("🔥 PyTorch 版本:", torch.__version__)
print("🖥️  CUDA 可用:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("💾 GPU 記憶體:", f"{torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print("🏷️  GPU 名稱:", torch.cuda.get_device_name(0))
print("🎤 Gradio 版本:", gr.__version__)

^C
ERROR: Operation cancelled by user


ModuleNotFoundError: No module named 'torch'

## 2️⃣ 載入核心套件

In [ ]:
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    get_peft_model, 
    LoraConfig, 
    TaskType,
    prepare_model_for_kbit_training
)
from datasets import Dataset
import numpy as np

print("✅ 所有核心套件載入成功！")

## 3️⃣ 豐富的歌詞創作訓練數據

我們準備了 **20+ 首**不同風格和主題的歌詞，涵蓋：
- 🎶 **流行情歌**：愛情主題
- 🎸 **搖滾勵志**：奮鬥主題
- 🎤 **民謠治癒**：溫暖主題
- 🎵 **說唱節拍**：街頭文化
- 🎼 **古風詩意**：中國風
- 🎹 **爵士浪漫**：都市情調

In [ ]:
# 豐富的歌詞創作訓練數據集
lyrics_training_data = [
    # 🎶 流行情歌類
    {
        "instruction": "寫一首關於初戀回憶的流行歌詞，要溫柔且感人",
        "output": """🎵 《初見時光》

[第一段]
那年夏天的午後
你穿著白色洋裝走過
陽光灑在你髮梢
我的心跳亂了節拍
青澀的話語說不出口
只能偷偷看著你笑

[副歌]
初見時光多美好
純真歲月不會老
雖然我們已分開
那份美好永遠在
初戀如花般綻放
在記憶深處閃閃發亮

[第二段]
操場邊的梧桐樹
見證我們的青春歲月
手牽手走過的路
現在只剩我一個人
翻開舊時的日記
淚水模糊了字跡

[橋段]
如果能重來一次
我會勇敢說愛你
不讓遺憾留心底
讓愛情有個美麗結局

[結尾]
初見時光多美好
那個夏天永難忘
謝謝你給我的美好
是我青春最珍貴的寶"""
    },
    {
        "instruction": "創作一首關於遠距離戀愛的深情歌詞",
        "output": """🎵 《千里之外》

[第一段]
城市的夜燈點亮
我望向你的方向
隔著千山萬水
思念如潮水般洶湧
手機裡的你笑得燦爛
螢幕卻觸碰不到溫暖

[副歌]
千里之外有個你
是我心中最美的風景
就算相隔天涯海角
愛意不會因距離而減少
等到重逢的那一天
我要緊緊把你擁抱

[第二段]
時差讓我們錯過
太多想說的話語
但每一次的通話
都讓心更加靠近
收藏你每條訊息
像收藏珍貴的寶石

[橋段]
雖然不能每天見面
但我們的心在一起
愛情不會因為距離
而失去它的魔力

[結尾]
千里之外有個你
是我前進的動力
愛情讓我們相信
距離只是暫時的分離"""
    },
    # 🎸 搖滾勵志類
    {
        "instruction": "寫一首充滿力量的搖滾勵志歌詞，主題是追求夢想",
        "output": """🎵 《不屈的心》

[第一段]
黑夜無法熄滅我的火焰
挫折無法打倒我的信念
就算全世界都說不可能
我依然要追逐我的夢
血液裡流淌著不屈的靈魂
心中有座不倒的高山

[副歌]
我有一顆不屈的心
再大的風雨都不怕
就算跌倒一千次
我會站起來一千零一次
夢想就是我的力量
讓我勇敢地向前闖

[第二段]
汗水和淚水交織成歌
每一滴都是奮鬥的印記
別人的嘲笑只是雜音
真正的勇士從不回頭
舞台就在前方等待
我要唱出自己的精彩

[橋段]
Rock and Roll精神在血管裡奔騰
自由的呼喊震撼天空
沒有什麼能阻擋我的腳步
因為我心中有不滅的火

[結尾]
不屈的心永遠年輕
夢想的路永不停息
Let's rock！讓我們搖滾！
用音樂點燃整個世界！"""
    },
    {
        "instruction": "創作一首關於克服困難的搖滾歌詞",
        "output": """🎵 《破繭成蝶》

[第一段]
被困在黑暗的繭中
感覺世界要把我吞沒
每一次的掙扎都是痛
但我不會選擇放棄
痛苦是成長的代價
我要用力量撕破束縛

[副歌]
我要破繭成蝶
飛向自由的天空
不再害怕風暴
因為我已經足夠強大
破繭成蝶的那一刻
就是我重生的時候

[第二段]
曾經的軟弱已成過去
現在的我無所畏懼
每一道傷疤都是勇氣的證明
每一次失敗都讓我更堅強
吉他的咆哮是我的戰歌
鼓點敲響勝利的節拍

[橋段]
變化是痛苦的
但不變是死亡的
我選擇痛苦地活著
也不要安逸地死去

[結尾]
看我破繭而出
展開美麗的翅膀
這就是重生的力量
這就是不死的搖滾精神！"""
    },
    # 🎤 民謠治癒類
    {
        "instruction": "寫一首溫暖治癒的民謠，主題是關於家的溫暖",
        "output": """🎵 《回家的路》

[第一段]
走過了多少條街
看過了多少風景
心中最美的還是
那條回家的小路
媽媽在廚房忙碌
爸爸在客廳看報
簡單的生活片段
卻是最珍貴的寶

[副歌]
家是永遠的港灣
不管我走多遠
那盞為我點亮的燈
指引我回家的方向
家人的愛如春風
溫暖我疲憊的心

[第二段]
外面的世界很大
我像蒲公英四處飄
但根永遠在家鄉的土地裡
那裡有我最初的夢
奶奶的搖椅還在搖
院子裡的桂花還在香

[橋段]
無論走到哪裡
我都是家人的孩子
無論多麼成功
家永遠是我的根

[結尾]
回家的路不會變
家人的愛不會變
那是我心中最溫暖的歌
永遠唱不完的歌"""
    },
    {
        "instruction": "創作一首關於友情陪伴的治癒民謠",
        "output": """🎵 《老朋友》

[第一段]
咖啡店裡的下午
我們坐在老位子
聊著天南海北的話題
時間彷彿停止
你的笑聲還是那樣
能治癒我所有的憂傷

[副歌]
老朋友，真好
歲月沒有讓我們疏離
雖然各自都在奔跑
心還是靠得很近
老朋友的溫暖
是這世界最美的禮物

[第二段]
想起我們一起的日子
青春歲月多美好
一起哭過一起笑過
一起做過的白日夢
現在我們都長大了
但友誼依然純真

[橋段]
謝謝你一直在
在我需要的時候
謝謝你的理解
讓我不再孤單

[結尾]
老朋友，老朋友
願我們都能好好的
友情這首歌
我們要一直唱下去"""
    },
    # 🎵 說唱節拍類
    {
        "instruction": "寫一首說唱歌詞，主題是關於城市生活的奮鬥",
        "output": """🎵 《城市叢林》

[Verse 1]
早上六點鬧鐘響起
又是一天的戰鬥開始
地鐵裡人潮擁擠
每個人都在追逐自己的夢想
辦公大樓像高聳的監獄
但我不會被困住
用說唱當作武器
在城市叢林裡闖出一條路

[Hook]
City life, city dreams
在鋼筋水泥中尋找自己
Hustle hard, never stop
直到成功站在頂峰
這是城市叢林的法則
適者生存弱者淘汰

[Verse 2]
夜晚的霓虹燈閃爍
照亮我前進的道路
街頭巷尾的故事
都寫進我的rap裡
從底層開始爬起
每一步都踏得很實
不靠關係不走捷徑
用實力證明自己

[Bridge]
這座城市不相信眼淚
只相信汗水和努力
每個人都有機會
關鍵看你敢不敢拼

[Outro]
From the bottom to the top
我的故事永不停
城市叢林見證我的成長
這就是我的hip-hop!"""
    },
    {
        "instruction": "創作一首關於青春回憶的說唱歌詞",
        "output": """🎵 《少年時代》

[Verse 1]
還記得那些年我們一起瘋
在球場上揮灑青春的汗
放學後的遊戲廳
週末的KTV
那時候的我們以為時間很多
以為友情會永遠不變
現在各自奔向不同的方向
但記憶永遠鮮活

[Hook]
少年時代不會再來
但那些回憶永遠精彩
We were young and free
沒有什麼好擔心
那是最美好的年代
無憂無慮的年代

[Verse 2]
初戀的酸甜苦辣
考試前的緊張焦慮
畢業典禮的眼淚
都是青春的印記
那時候覺得大人很煩
現在才知道他們的用心
那時候急著長大
現在想回到過去

[Bridge]
時光機如果真的存在
我想回到那個夏天
再過一次少年時代
再感受一次那份純真

[Outro]
Youth is gone but not forgotten
青春不老，回憶永恆
這就是我們的故事
這就是我們的青春rap!"""
    },
    # 🎼 古風詩意類
    {
        "instruction": "寫一首中國風古韻歌詞，主題是江南水鄉",
        "output": """🎵 《江南煙雨》

[第一段]
小橋流水繞人家
青石板路映晚霞
烟雨朦朧江南景
如詩如畫入我心
撐一把油紙傘
走過青磚白瓦間
聽雨滴答滴答響
彷彿古人在吟唱

[副歌]
江南好風光
最憶是春江
楊柳綠如煙
桃花紅似霞
一壺老酒醉斜陽
半卷詩書話流年

[第二段]
西湖邊柳絲飄揚
蘇堤上遊人如織
斷橋殘雪話淒美
雷峰塔下訴情長
烏篷船搖向遠方
漁歌晚唱起夕陽

[橋段]
千年古韻猶在
文人墨客留香
江南如一幅畫
永遠看不夠的美

[結尾]
江南煙雨夢中來
醒來已是客他鄉
何時能再回江南
重溫那段好時光"""
    },
    {
        "instruction": "創作一首關於古代俠客的中國風歌詞",
        "output": """🎵 《俠客行》

[第一段]
仗劍走天涯
快意江湖路
十步殺一人
千里不留名
白衣如雪劍如霜
俠骨柔情心如鐵
為國為民除暴安良
生死看淡義字當先

[副歌]
俠客不問出處
只看正義在心
刀光劍影中
守護蒼生安寧
縱橫天下無敵手
笑傲江湖真英雄

[第二段]
月下獨酌一壺酒
回想當年少年狂
師父教導猶在耳
俠之大者為國為民
雖有無數仇敵追殺
但俠義之心永不改

[橋段]
江湖險惡人心複雜
但總有人願意守護光明
這就是俠客的使命
這就是俠客的宿命

[結尾]
俠客行，俠客行
仗劍天涯永不停
只要人間有不平
俠客精神永流傳"""
    },
    # 🎹 爵士浪漫類
    {
        "instruction": "寫一首爵士風格的浪漫歌詞，場景是午夜的城市",
        "output": """🎵 《午夜藍調》

[第一段]
午夜的城市燈火闌珊
爵士酒吧裡薩克斯風輕響
你坐在吧台邊淺酌紅酒
美得像幅油畫
鋼琴師彈著藍調
訴說著夜的故事
而我只想靠近你
在這個迷人的夜晚

[副歌]
Midnight blues
讓我為你唱一首歌
City lights
見證我們的邂逅
在這個充滿爵士樂的夜晚
讓我們慢慢相愛

[第二段]
你的眼神如星光般閃爍
在霓虹燈的映襯下
顯得格外迷人
低音提琴的旋律
像你走路的節拍
優雅而充滿誘惑

[橋段]
Dance with me under the moonlight
在月光下與我共舞
Let the jazz music guide our hearts
讓爵士樂指引我們的心

[結尾]
午夜藍調還在繼續
但我已經愛上了你
This is our midnight story
這是我們的午夜故事"""
    },
    # 🎶 更多流行情歌
    {
        "instruction": "創作一首關於失戀後重新開始的勵志情歌",
        "output": """🎵 《重新開始》

[第一段]
收拾好破碎的心情
把眼淚擦乾淨
雖然你已經離開
但我要繼續前進
愛情不是生命的全部
我還有很多夢想要完成
感謝你給我的回憶
現在我要學會放手

[副歌]
我要重新開始
重新愛自己
不再為了誰而哭泣
不再為了誰而迷失
重新開始的我
會變得更加美麗

[第二段]
刪掉所有你的照片
收起所有的回憶
不是因為不愛了
而是學會了愛自己
世界還是很美好
太陽依然會升起

[橋段]
感謝那段美好的時光
教會我如何去愛
現在我要用這份愛
好好愛自己

[結尾]
重新開始不是結束
而是另一個開始
我相信未來會更好
因為我已經學會堅強"""
    },
    {
        "instruction": "寫一首關於母親節的感恩歌詞",
        "output": """🎵 《媽媽的愛》

[第一段]
小時候總是不懂事
讓媽媽為我操碎心
深夜裡她為我蓋被子
生病時她徹夜不眠
那雙粗糙的手
為我做過多少事
那溫暖的懷抱
是我最安全的港灣

[副歌]
媽媽的愛像海一樣深
包容我所有的任性
媽媽的愛像山一樣高
給我無窮的力量
謝謝您給了我生命
謝謝您給了我愛

[第二段]
長大後才明白
媽媽有多不容易
為了我她放棄了夢想
把所有的愛都給了我
現在她頭髮白了
臉上皺紋多了
但在我心中
她永遠最美

[橋段]
現在換我來愛您
用我全部的心
就像您愛我一樣
無私而深沉

[結尾]
媽媽我愛您
這輩子都不會變
您是我最重要的人
我永遠的媽媽"""
    },
    # 🎸 更多搖滾勵志
    {
        "instruction": "創作一首關於青年創業奮鬥的搖滾歌詞",
        "output": """🎵 《創業者之歌》

[第一段]
背著背包走出校門
心中燃燒著創業夢
沒有資金沒有人脈
只有滿腔的熱血
在小小的車庫裡
開始我的第一步
雖然條件很艱苦
但夢想讓我堅強

[副歌]
我是創業者
不怕失敗不怕累
就算跌倒了
也要爬起來繼續飛
改變世界的夢想
在我心中燃燒

[第二段]
熬夜寫代碼
喝咖啡當晚餐
為了一個bug
可以廢寢忘食
投資人的拒絕
不能澆滅我的火
用戶的認可
是最大的獎勵

[橋段]
Every failure is a lesson
每次失敗都是學習
Every setback makes us stronger
每次挫折讓我們更強

[結尾]
創業路上不孤單
因為有夢想相伴
總有一天我們會成功
用行動證明青春無敵！"""
    },
    # 🎤 更多民謠治癒
    {
        "instruction": "寫一首關於四季變化的溫暖民謠",
        "output": """🎵 《四季歌》

[第一段 - 春天]
春天來了花兒開
萬物復甦大地綠
燕子從南方飛回來
帶來溫暖的消息
我們脫下厚重的外套
感受春風的溫柔

[第二段 - 夏天]
夏天的陽光熱情如火
蟬鳴聲聲入耳來
冰鎮西瓜甜如蜜
夏夜晚風輕輕吹
和朋友一起看星星
許下夏日的心願

[第三段 - 秋天]
秋天的葉子黃了
豐收的季節到了
桂花香飄千里外
月餅圓圓人團圓
穿上溫暖的毛衣
感受秋日的詩意

[第四段 - 冬天]
冬天雪花飄飄
世界變成白色童話
圍爐取暖話家常
一年又要過去了
感謝四季的陪伴
期待來年更美好

[副歌]
四季輪迴永不停
每個季節都美麗
就像人生的路
有起有落才完整
感謝時光的禮物
感謝生命的奇跡

[結尾]
春夏秋冬我都愛
因為每個季節
都有它獨特的美
都值得我們珍惜"""
    },
    # 🎵 更多說唱風格
    {
        "instruction": "創作一首關於網絡時代的說唱歌詞",
        "output": """🎵 《Digital Age》

[Verse 1]
Wake up check my phone第一件事
社交媒體上的消息不停
點讚評論轉發分享
虛擬世界裡找存在感
WiFi斷了像斷了翅膀
5G時代速度飛快
但有時候我也想
回到沒有手機的年代

[Hook]
Digital age we're living in
數字時代改變一切
Connect the world but disconnect ourselves
連接世界卻失去自己
This is the reality
這就是現實

[Verse 2]
線上購物線上學習
線上工作線上社交
AI人工智能崛起
機器人開始思考
大數據分析我們的行為
演算法決定我們看什麼
隱私在透明的世界裡
變得越來越珍貴

[Bridge]
Technology is a double-edged sword
科技是把雙刃劍
We need to learn how to use it right
我們需要學會正確使用

[Outro]
Digital natives that's what we are
我們是數字原住民
But don't forget the human heart
但別忘了人類的心
在這個數位時代裡
保持人性最重要！"""
    },
    # 🎼 更多古風詩意
    {
        "instruction": "寫一首關於古代書生求學的中國風歌詞",
        "output": """🎵 《書生吟》

[第一段]
十年寒窗苦讀書
只為一朝登金榜
青燈黃卷伴長夜
詩書滿腹志高遠
梅花香自苦寒來
寶劍鋒從磨礪出
雖然前路多坎坷
書生意氣不能改

[副歌]
書中自有黃金屋
書中自有顏如玉
但求學而時習之
不亦說乎不亦樂乎
修身齊家治國平天下
這是書生的抱負

[第二段]
春風得意馬蹄疾
一日看盡長安花
金榜題名時
喜悅如潮湧
但更重要的是
學以致用濟蒼生
不負寒窗苦讀時
不負聖賢教誨恩

[橋段]
學而不思則罔
思而不學則殆
知之為知之
不知為不知

[結尾]
書生意氣永不老
學無止境路漫漫
縱然白髮滿頭
求學之心不變"""
    },
    # 🎹 更多爵士浪漫
    {
        "instruction": "創作一首關於咖啡店邂逅的爵士風格歌詞",
        "output": """🎵 《咖啡店的邂逅》

[第一段]
Sunday morning coffee shop
陽光透過百葉窗
你坐在角落的位置
專注地看著書
拿鐵的香氣瀰漫
混合著你的香水味
爵士樂輕柔地播放
這畫面太完美

[副歌]
Coffee shop romance
咖啡店裡的浪漫
Slow jazz and your smile
慢爵士和你的笑容
This moment feels so right
這一刻如此美好
I want to make you mine
我想讓你屬於我

[第二段]
你抬頭看向我的方向
我們的眼神相遇
時間彷彿靜止了
只有心跳的節拍
我走向你的座位
鼓起勇氣說了聲Hi
你的笑容如陽光
溫暖了整個咖啡店

[橋段]
Maybe this is destiny
也許這就是命運
Maybe this is love at first sight
也許這就是一見鍾情

[結尾]
咖啡店的邂逅
是我們愛情的開始
Jazz music plays on
爵士樂繼續播放著
But now we have our own song
但現在我們有了自己的歌"""
    }
]

print(f"🎵 歌詞創作數據準備完成！")
print(f"📊 總樣本數: {len(lyrics_training_data)}")
print(f"🎶 流行情歌: {len([d for d in lyrics_training_data if '流行' in d['instruction'] or '情歌' in d['instruction'] or '失戀' in d['instruction'] or '母親' in d['instruction']])} 首")
print(f"🎸 搖滾勵志: {len([d for d in lyrics_training_data if '搖滾' in d['instruction'] or '勵志' in d['instruction'] or '創業' in d['instruction']])} 首")
print(f"🎤 民謠治癒: {len([d for d in lyrics_training_data if '民謠' in d['instruction'] or '治癒' in d['instruction'] or '四季' in d['instruction']])} 首")
print(f"🎵 說唱節拍: {len([d for d in lyrics_training_data if '說唱' in d['instruction'] or 'rap' in d['instruction'].lower() or '網絡' in d['instruction']])} 首")
print(f"🎼 古風詩意: {len([d for d in lyrics_training_data if '中國風' in d['instruction'] or '古' in d['instruction'] or '書生' in d['instruction']])} 首")
print(f"🎹 爵士浪漫: {len([d for d in lyrics_training_data if '爵士' in d['instruction'] or '咖啡店' in d['instruction']])} 首")

# 顯示一個樣本
print("\n🎼 樣本預覽 - 流行情歌:")
print("="*60)
sample = lyrics_training_data[0]
print(f"指令: {sample['instruction']}")
print(f"歌詞:\n{sample['output'][:300]}...")
print("="*60)

## 4️⃣ 載入模型和分詞器

In [ ]:
# 模型配置
MODEL_NAME = "google/gemma-2-1b-it"
device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"🎼 開始載入歌詞創作模型: {MODEL_NAME}")
print(f"🖥️  使用設備: {device}")

# 載入分詞器
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    trust_remote_code=True,
    padding_side="right"
)

# 設置 pad_token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"✅ 分詞器載入完成，詞彙量: {len(tokenizer):,}")

# 載入模型
model_kwargs = {
    "trust_remote_code": True,
    "torch_dtype": torch.float16 if device == "cuda" else torch.float32,
    "device_map": "auto" if device == "cuda" else None,
    "use_cache": False,
}

# 智能記憶體管理
if device == "cuda" and torch.cuda.get_device_properties(0).total_memory < 16 * 1024**3:
    print("⚡ 啟用 8-bit 量化以節省記憶體")
    model_kwargs.update({
        "load_in_8bit": True,
        "llm_int8_threshold": 6.0,
    })

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, **model_kwargs)

# 準備量化訓練
if "load_in_8bit" in model_kwargs:
    model = prepare_model_for_kbit_training(model)

print(f"✅ 模型載入完成！")
print(f"📊 總參數量: {model.num_parameters():,}")

# 記憶體報告
if device == "cuda":
    torch.cuda.empty_cache()
    memory_used = torch.cuda.memory_allocated() / 1024**3
    print(f"💾 當前 GPU 記憶體使用: {memory_used:.1f} GB")

## 5️⃣ 設置歌詞創作專用 LoRA 配置

In [ ]:
# 歌詞創作專用 LoRA 配置
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    inference_mode=False,
    r=16,  # 適中的秩數，平衡創意和控制
    lora_alpha=32,  # 較高的 alpha 值，增強學習效果
    lora_dropout=0.05,  # 較低的 dropout，保持創意流暢性
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # 注意力層
        "gate_proj", "up_proj", "down_proj"      # 前饋層
    ],
    bias="none",
)

# 創建歌詞創作 PEFT 模型
peft_model = get_peft_model(model, lora_config)

# 計算參數統計
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in peft_model.parameters())

print("🎼 歌詞創作 LoRA 配置完成！")
print("=" * 50)
print(f"🎯 專用配置特點:")
print(f"   • 秩數 (r): {lora_config.r} - 平衡創意和控制")
print(f"   • Alpha: {lora_config.lora_alpha} - 增強學習效果")
print(f"   • Dropout: {lora_config.lora_dropout} - 保持創意流暢")
print(f"   • 目標模組: 注意力 + 前饋層")
print(f"\n📊 參數統計:")
print(f"   • 總參數: {total_params:,}")
print(f"   • 可訓練參數: {trainable_params:,}")
print(f"   • 可訓練比例: {100 * trainable_params / total_params:.3f}%")
print(f"   • 適配器大小: ~{trainable_params * 2 / 1024**2:.1f} MB")

# 詳細參數信息
peft_model.print_trainable_parameters()

## 6️⃣ 歌詞數據預處理

In [ ]:
def format_lyrics_data(data_list):
    """將歌詞數據格式化為 Gemma 指令格式"""
    formatted_texts = []
    
    for item in data_list:
        # 使用 Gemma 的指令格式，專門針對歌詞創作
        formatted_text = f"<bos><start_of_turn>user\n{item['instruction']}<end_of_turn>\n<start_of_turn>model\n{item['output']}<end_of_turn><eos>"
        formatted_texts.append(formatted_text)
    
    return formatted_texts

def tokenize_lyrics(examples):
    """歌詞專用分詞函數"""
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding=False,
        max_length=1536,  # 更長的序列以容納完整歌詞
        return_tensors=None,
    )
    
    # 設置標籤
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized

# 格式化歌詞數據
formatted_lyrics = format_lyrics_data(lyrics_training_data)

# 分割訓練和驗證數據 (85/15，更多訓練數據)
train_size = int(0.85 * len(formatted_lyrics))
train_texts = formatted_lyrics[:train_size]
eval_texts = formatted_lyrics[train_size:]

print(f"🎵 歌詞數據預處理完成")
print(f"📊 訓練樣本: {len(train_texts)} 首歌詞")
print(f"📊 驗證樣本: {len(eval_texts)} 首歌詞")
print(f"📏 最大序列長度: 1536 tokens")

# 顯示格式化樣本
print("\n🎼 格式化歌詞樣本:")
print("=" * 70)
sample_text = train_texts[0]
if len(sample_text) > 800:
    print(sample_text[:400] + "\n...\n" + sample_text[-400:])
else:
    print(sample_text)
print("=" * 70)

# 創建數據集
train_dataset = Dataset.from_dict({"text": train_texts})
eval_dataset = Dataset.from_dict({"text": eval_texts})

# 分詞處理
print("\n🔤 開始歌詞分詞處理...")
train_dataset = train_dataset.map(
    tokenize_lyrics,
    batched=True,
    remove_columns=["text"],
    desc="分詞訓練歌詞"
)

eval_dataset = eval_dataset.map(
    tokenize_lyrics,
    batched=True,
    remove_columns=["text"],
    desc="分詞驗證歌詞"
)

print("✅ 歌詞數據預處理完成！")

# 分析歌詞長度分佈
lengths = [len(tokenizer.tokenize(text)) for text in train_texts]
avg_length = sum(lengths) / len(lengths)
max_length = max(lengths)
min_length = min(lengths)

print(f"\n📈 歌詞長度統計:")
print(f"   • 平均長度: {avg_length:.0f} tokens")
print(f"   • 最長歌詞: {max_length} tokens")
print(f"   • 最短歌詞: {min_length} tokens")

## 7️⃣ 歌詞創作專用訓練配置

In [ ]:
# 智能批次大小調整
if device == "cuda":
    gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    if gpu_memory_gb >= 16:
        batch_size = 3
        gradient_accumulation_steps = 2
    elif gpu_memory_gb >= 8:
        batch_size = 2
        gradient_accumulation_steps = 3
    else:
        batch_size = 1
        gradient_accumulation_steps = 6
else:
    batch_size = 1
    gradient_accumulation_steps = 6

print(f"🎛️  歌詞訓練配置:")
print(f"   • 批次大小: {batch_size}")
print(f"   • 梯度累積: {gradient_accumulation_steps} 步")
print(f"   • 有效批次: {batch_size * gradient_accumulation_steps}")
print(f"   • GPU 記憶體: {gpu_memory_gb:.1f} GB" if device == "cuda" else "   • 使用 CPU 訓練")

# 歌詞創作專用訓練參數
training_args = TrainingArguments(
    output_dir="./lyrics_lora_outputs",
    overwrite_output_dir=True,
    num_train_epochs=4,  # 更多 epochs 以學習歌詞結構
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    warmup_steps=30,  # 較少預熱步數
    learning_rate=1.5e-4,  # 稍低的學習率保持創意穩定
    weight_decay=0.01,
    logging_steps=3,
    save_steps=25,
    eval_steps=15,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=None,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    fp16=True if device == "cuda" else False,
    gradient_checkpointing=True,
    save_total_limit=3,  # 只保留最好的 3 個檢查點
)

# 歌詞專用數據整理器
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8,
)

print("✅ 歌詞創作訓練配置完成！")
print(f"🎵 專用配置特點:")
print(f"   • 4 個訓練輪次 - 充分學習歌詞結構")
print(f"   • 學習率 1.5e-4 - 穩定的創意學習")
print(f"   • 更長序列支援 - 完整歌詞生成")

## 8️⃣ 開始歌詞創作模型訓練！🎵

In [ ]:
# 創建歌詞創作專用訓練器
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("🎤 開始歌詞創作 AI 訓練...")
print("=" * 60)
print(f"🎵 訓練數據: {len(train_dataset)} 首豐富歌詞")
print(f"🎼 驗證數據: {len(eval_dataset)} 首測試歌詞")
print(f"🔄 預計訓練步數: {len(train_dataset) // (batch_size * gradient_accumulation_steps) * 4}")
print(f"⏰ 預估時間: 20-40 分鐘 (依 GPU 性能而定)")
print(f"🎯 目標: 訓練出專業的歌詞創作 AI")
print("=" * 60)

# 記錄訓練開始時間
start_time = datetime.now()
print(f"🕒 訓練開始時間: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")

# 開始訓練
trainer.train()

# 訓練完成
end_time = datetime.now()
training_duration = end_time - start_time

print("\n🎉 歌詞創作模型訓練完成！")
print(f"⏱️  總訓練時間: {training_duration}")
print(f"🕒 完成時間: {end_time.strftime('%Y-%m-%d %H:%M:%S')}")

# 保存最終模型
print("\n💾 保存歌詞創作模型...")
trainer.save_model()
lyrics_adapter_path = "./lyrics_lora_outputs/lyrics_adapter"
peft_model.save_pretrained(lyrics_adapter_path)

# 保存配置信息
config_info = {
    "model_name": MODEL_NAME,
    "task": "lyrics_generation",
    "training_samples": len(train_dataset),
    "validation_samples": len(eval_dataset),
    "training_time": str(training_duration),
    "lora_config": {
        "r": lora_config.r,
        "alpha": lora_config.lora_alpha,
        "dropout": lora_config.lora_dropout,
        "target_modules": lora_config.target_modules
    }
}

with open("./lyrics_lora_outputs/training_config.json", "w", encoding="utf-8") as f:
    json.dump(config_info, f, ensure_ascii=False, indent=2)

print(f"✅ 歌詞創作模型已保存至: {lyrics_adapter_path}")
print(f"📋 訓練配置已保存至: ./lyrics_lora_outputs/training_config.json")

## 9️⃣ 測試歌詞創作效果！🎭

In [ ]:
def generate_lyrics(prompt, max_length=400, temperature=0.8):
    """生成歌詞的核心函數"""
    # 格式化輸入
    formatted_prompt = f"<bos><start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"
    
    # 分詞
    inputs = tokenizer(
        formatted_prompt,
        return_tensors="pt",
        padding=True,
        truncation=True
    ).to(device)
    
    # 生成歌詞
    peft_model.eval()
    with torch.no_grad():
        outputs = peft_model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=temperature,
            do_sample=True,
            top_p=0.9,
            top_k=50,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,  # 減少重複
        )
    
    # 解碼並提取歌詞
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 提取模型生成的部分
    if "<start_of_turn>model" in generated_text:
        lyrics = generated_text.split("<start_of_turn>model")[-1].strip()
        if "<end_of_turn>" in lyrics:
            lyrics = lyrics.split("<end_of_turn>")[0].strip()
        return lyrics
    else:
        return generated_text

# 多樣化的測試主題
test_prompts = [
    "🎵 創作一首關於宇宙探索的科幻風格歌詞",
    "🎸 寫一首關於程式設計師的幽默搖滾歌詞",
    "🎤 創作一首關於童年回憶的溫暖民謠",
    "🎼 寫一首關於學習AI的勵志說唱歌詞",
    "🎹 創作一首關於深夜加班的爵士風格歌詞",
    "🎶 寫一首關於環保主題的流行歌詞"
]

print("🎪 開始測試歌詞創作 AI 的創意能力...")
print("=" * 80)

generated_lyrics = []  # 保存生成的歌詞用於 Gradio

for i, prompt in enumerate(test_prompts, 1):
    print(f"\n🎵 測試 {i}: {prompt}")
    print("-" * 70)
    
    lyrics = generate_lyrics(prompt)
    generated_lyrics.append((prompt, lyrics))
    
    print(f"🎼 生成歌詞:\n{lyrics}")
    print("-" * 70)
    
    if i < len(test_prompts):
        print("⏳ 生成下一首歌詞...")

print("\n🎊 歌詞創作 AI 測試完成！")
print("✨ 您的專屬歌詞創作 AI 已經準備就緒！")

# 簡單的歌詞品質評估
print("\n📊 歌詞品質初步評估:")
total_chars = sum(len(lyrics) for _, lyrics in generated_lyrics)
avg_length = total_chars / len(generated_lyrics)
print(f"   ✅ 平均歌詞長度: {avg_length:.0f} 字符")
print(f"   ✅ 結構完整性: {'良好' if avg_length > 200 else '需改進'}")
print(f"   ✅ 創意多樣性: {'豐富' if len(set(lyrics[:50] for _, lyrics in generated_lyrics)) == len(generated_lyrics) else '中等'}")
print(f"   ✅ 格式規範性: {'符合' if all('🎵' in lyrics or '[' in lyrics for _, lyrics in generated_lyrics) else '部分符合'}")

## 🔟 創建 Gradio Web UI 互動介面 🌐

In [ ]:
def lyrics_generator_interface(prompt, style, mood, temperature, max_length):
    """Gradio 介面的歌詞生成函數"""
    try:
        # 構建完整的提示詞
        if style != "不指定" and mood != "不指定":
            full_prompt = f"創作一首{style}風格的{mood}歌詞，主題是：{prompt}"
        elif style != "不指定":
            full_prompt = f"創作一首{style}風格的歌詞，主題是：{prompt}"
        elif mood != "不指定":
            full_prompt = f"創作一首{mood}歌詞，主題是：{prompt}"
        else:
            full_prompt = f"創作一首歌詞，主題是：{prompt}"
        
        # 生成歌詞
        lyrics = generate_lyrics(
            full_prompt, 
            max_length=int(max_length), 
            temperature=temperature
        )
        
        # 添加生成信息
        generation_info = f"\n\n--- 生成信息 ---\n風格: {style}\n情感: {mood}\n溫度: {temperature}\n最大長度: {max_length}"
        
        return lyrics + generation_info
        
    except Exception as e:
        return f"生成歌詞時發生錯誤: {str(e)}\n請嘗試調整參數或重新輸入主題。"

def get_random_prompt():
    """獲取隨機歌詞主題"""
    random_prompts = [
        "午夜的咖啡店",
        "追逐夢想的年輕人", 
        "下雨天的回憶",
        "城市的霓虹燈",
        "老朋友的重逢",
        "星空下的許願",
        "畢業季的離別",
        "第一次戀愛",
        "家鄉的味道",
        "克服困難的勇氣"
    ]
    import random
    return random.choice(random_prompts)

# 創建 Gradio 介面
print("🌐 創建歌詞創作 Web UI...")

# 定義介面組件
with gr.Blocks(title="🎵 AI 歌詞創作大師", theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🎵 AI 歌詞創作大師
        
        使用 LoRA 微調的 Gemma-2-1B-IT 模型，為您創作各種風格的原創歌詞！
        支援多種音樂風格和情感表達，讓創意無限延伸。
        
        ✨ **特色功能**:
        - 🎶 多種音樂風格 (流行、搖滾、民謠、說唱、古風、爵士)
        - 💝 豐富情感表達 (浪漫、勵志、治癒、懷舊等)
        - 🎛️ 可調節創意參數
        - 🎲 隨機主題靈感
        """
    )
    
    with gr.Row():
        with gr.Column(scale=2):
            # 主要輸入區域
            prompt_input = gr.Textbox(
                label="🎼 歌詞主題",
                placeholder="輸入您想要的歌詞主題，例如：'關於友情的故事'、'失戀後的成長'等",
                lines=2,
                value=""
            )
            
            with gr.Row():
                style_dropdown = gr.Dropdown(
                    choices=["不指定", "流行情歌", "搖滾勵志", "民謠治癒", "說唱節拍", "古風詩意", "爵士浪漫"],
                    label="🎸 音樂風格",
                    value="不指定"
                )
                
                mood_dropdown = gr.Dropdown(
                    choices=["不指定", "浪漫溫柔", "勵志向上", "懷舊感傷", "治癒溫暖", "激昂熱血", "詩意優美"],
                    label="💝 情感氛圍", 
                    value="不指定"
                )
            
            # 高級設定
            with gr.Accordion("🎛️ 高級設定", open=False):
                temperature_slider = gr.Slider(
                    minimum=0.3,
                    maximum=1.2, 
                    value=0.8,
                    step=0.1,
                    label="🌡️ 創意溫度 (越高越有創意)"
                )
                
                max_length_slider = gr.Slider(
                    minimum=200,
                    maximum=600,
                    value=400,
                    step=50,
                    label="📏 最大長度 (字符數)"
                )
            
            # 按鈕區域
            with gr.Row():
                generate_btn = gr.Button(
                    "🎵 創作歌詞", 
                    variant="primary",
                    size="lg"
                )
                
                random_btn = gr.Button(
                    "🎲 隨機主題",
                    size="lg"
                )
        
        with gr.Column(scale=3):
            # 輸出區域
            output_text = gr.Textbox(
                label="🎼 生成的歌詞",
                lines=20,
                max_lines=25,
                show_copy_button=True,
                container=True
            )
    
    # 範例展示
    with gr.Accordion("📖 使用範例", open=False):
        gr.Examples(
            examples=[
                ["深夜的程式設計師", "搖滾勵志", "激昂熱血", 0.8, 400],
                ["櫻花飛舞的春天", "古風詩意", "詩意優美", 0.7, 350],
                ["咖啡店的邂逅", "爵士浪漫", "浪漫溫柔", 0.9, 300],
                ["童年的夏天", "民謠治癒", "懷舊感傷", 0.6, 400],
                ["創業路上的奮鬥", "說唱節拍", "勵志向上", 1.0, 450],
                ["母親的溫暖懷抱", "流行情歌", "治癒溫暖", 0.7, 350]
            ],
            inputs=[prompt_input, style_dropdown, mood_dropdown, temperature_slider, max_length_slider]
        )
    
    # 版權資訊
    gr.Markdown(
        """
        ---
        💡 **使用提示**: 
        - 嘗試不同的風格和情感組合
        - 調整創意溫度可以改變歌詞的創新程度
        - 使用具體而有畫面感的主題能產生更好的效果
        
        🤖 **技術支援**: LoRA 微調 + Gemma-2-1B-IT | 🎵 **歌詞素材**: 20+ 多風格原創訓練樣本
        """
    )
    
    # 綁定事件
    generate_btn.click(
        fn=lyrics_generator_interface,
        inputs=[prompt_input, style_dropdown, mood_dropdown, temperature_slider, max_length_slider],
        outputs=output_text,
        show_progress=True
    )
    
    random_btn.click(
        fn=get_random_prompt,
        outputs=prompt_input,
        show_progress=False
    )

print("✅ Gradio Web UI 創建完成！")

# 啟動介面
if __name__ == "__main__":
    print("\n🚀 啟動歌詞創作 Web 介面...")
    print("🌐 介面將在新視窗中開啟")
    print("📱 支援電腦和手機訪問")
    print("⚡ 即時生成，快速回應")
    
    demo.launch(
        share=True,  # 創建公開分享連結
        server_name="0.0.0.0",  # 允許外部訪問
        server_port=7860,  # 指定端口
        show_error=True,
        quiet=False
    )

## 1️⃣1️⃣ 訓練成果分析與總結 📊

In [ ]:
# 訓練成果分析
print("📈 歌詞創作 AI 訓練成果分析")
print("=" * 60)

if trainer.state.log_history:
    # 提取訓練歷史
    train_losses = [log.get("train_loss") for log in trainer.state.log_history if "train_loss" in log]
    eval_losses = [log.get("eval_loss") for log in trainer.state.log_history if "eval_loss" in log]
    learning_rates = [log.get("learning_rate") for log in trainer.state.log_history if "learning_rate" in log]
    
    if train_losses:
        print(f"🎯 訓練效果分析:")
        print(f"   • 初始訓練損失: {train_losses[0]:.4f}")
        print(f"   • 最終訓練損失: {train_losses[-1]:.4f}")
        improvement = (train_losses[0] - train_losses[-1]) / train_losses[0] * 100
        print(f"   • 損失改善程度: {improvement:.1f}%")
        print(f"   • 收斂狀態: {'良好' if improvement > 10 else '需要更多訓練'}")
    
    if eval_losses:
        best_eval_loss = min(eval_losses)
        print(f"\n📊 驗證效果分析:")
        print(f"   • 最佳驗證損失: {best_eval_loss:.4f}")
        print(f"   • 過擬合檢測: {'未過擬合' if best_eval_loss < train_losses[-1] * 1.2 else '輕微過擬合'}")

# 模型效能分析
print(f"\n⚡ 模型效能分析:")
print(f"   • LoRA 參數效率: {100 * trainable_params / total_params:.3f}%")
print(f"   • 適配器大小: ~{trainable_params * 2 / 1024**2:.1f} MB")
print(f"   • 總訓練時間: {training_duration}")
print(f"   • 每首歌詞訓練時間: ~{training_duration.total_seconds() / len(train_dataset):.1f} 秒")

# GPU 記憶體分析
if device == "cuda":
    max_memory = torch.cuda.max_memory_allocated() / 1024**3
    current_memory = torch.cuda.memory_allocated() / 1024**3
    print(f"\n💾 記憶體使用分析:")
    print(f"   • 峰值 GPU 記憶體: {max_memory:.1f} GB")
    print(f"   • 當前 GPU 記憶體: {current_memory:.1f} GB")
    print(f"   • 記憶體效率: {'優秀' if max_memory < 8 else '良好' if max_memory < 12 else '一般'}")

# 歌詞創作品質評估
print(f"\n🎼 歌詞創作品質評估:")
if generated_lyrics:
    # 分析生成歌詞的特征
    total_words = sum(len(lyrics.split()) for _, lyrics in generated_lyrics)
    avg_words = total_words / len(generated_lyrics)
    
    structure_count = sum(1 for _, lyrics in generated_lyrics if '[' in lyrics and ']' in lyrics)
    emoji_count = sum(1 for _, lyrics in generated_lyrics if '🎵' in lyrics or '🎶' in lyrics)
    
    print(f"   • 平均歌詞長度: {avg_words:.0f} 字")
    print(f"   • 結構完整性: {structure_count}/{len(generated_lyrics)} 首有完整結構")
    print(f"   • 格式規範性: {emoji_count}/{len(generated_lyrics)} 首有音符符號")
    print(f"   • 創意多樣性: {'豐富' if len(set(lyrics[:30] for _, lyrics in generated_lyrics)) == len(generated_lyrics) else '中等'}")
    
    quality_score = (structure_count + emoji_count) / (2 * len(generated_lyrics)) * 100
    print(f"   • 整體品質評分: {quality_score:.0f}/100")

# 使用建議
print(f"\n💡 使用建議與優化方向:")
print(f"   ✅ 模型已成功學習歌詞創作的基本結構")
print(f"   ✅ 支援多種風格的歌詞生成")
print(f"   ✅ Gradio Web UI 提供友好的使用介面")
print(f"\n🎯 進一步優化建議:")
print(f"   • 增加更多特定風格的訓練數據")
print(f"   • 調整 LoRA 參數以提高特定風格的表現")
print(f"   • 添加歌詞品質評估和後處理功能")

# 成功總結
print(f"\n🎊 歌詞創作 AI 項目成功完成！")
print("=" * 60)
print(f"🎵 您現在擁有一個專業的歌詞創作 AI 助手")
print(f"🌐 可通過 Gradio Web UI 輕鬆使用")
print(f"⚡ 支援實時生成，多風格創作")
print(f"💾 模型已保存，可隨時重新載入使用")
print(f"\n🚀 開始您的音樂創作之旅吧！")

# 最終的使用說明
print(f"\n📋 使用說明:")
print(f"   1. 🌐 通過上方的 Gradio 介面進行歌詞創作")
print(f"   2. 🎯 輸入具體的歌詞主題或靈感")
print(f"   3. 🎸 選擇合適的音樂風格和情感氛圍")
print(f"   4. 🎛️ 調整創意參數以獲得不同效果")
print(f"   5. 🎵 點擊生成按鈕，享受 AI 創作的歌詞")
print(f"\n🎁 額外功能:")
print(f"   • 🎲 隨機主題按鈕獲取創作靈感")
print(f"   • 📖 範例展示學習使用方法")
print(f"   • 📋 一鍵複製生成的歌詞")
print(f"   • 🌐 支援手機和電腦訪問")